# Some Goals for this hack 

Some context:  I did this hacking on Saturday, October 25, 2014 for [BiB 2014](http://booksinbrowsers.org/2014-program/) hack day.

I wanted to learn a bit how about epub 2 & 3, specifically how to use Python to read and write epub files.

In [ ]:
# enumerate ebooklib types
import ebooklib

print (ebooklib.VERSION)

EBOOKLIB_TYPES = dict([(getattr(ebooklib,n), n) for n in dir(ebooklib) if n.startswith('ITEM_')])
EBOOKLIB_TYPES

In [ ]:
from itertools import islice
from ebooklib import epub

book = epub.read_epub('Oral_Literature_in_Africa.epub')

for (i, item) in enumerate(islice(book.get_items(),None)):
    print i, item


In [ ]:
# let's iterate through to see what we can read

from itertools import islice
from ebooklib import epub

book = epub.read_epub('Oral_Literature_in_Africa.epub')

k = book.get_items()

In [ ]:
item = k.next()
item

In [ ]:
import ebooklib
ebooklib.ITEM_STYLE

In [ ]:
(item.file_name, item.get_type())

In [ ]:
from itertools import islice

from ebooklib import epub

book = epub.read_epub('invisible_eink/invisible_eink_widget.epub')

for item in islice(book.get_items(),None):
    print item


In [ ]:
from ebooklib import epub

book = epub.EpubBook()

# set metadata
book.set_identifier('id123456')
book.set_title('Sample book')
book.set_language('en')

book.add_author('Author Authorowski')
book.add_author('Danko Bananko', file_as='Gospodin Danko Bananko', role='ill', uid='coauthor')

# create chapter
c1 = epub.EpubHtml(title='Intro', file_name='chap_01.xhtml', lang='hr')
c1.content=u'<h1>Intro heading</h1><p>Žaba je skočila u baru.</p>'

# add chapter
book.add_item(c1)

# define Table Of Contents
book.toc = (epub.Link('chap_01.xhtml', 'Introduction', 'intro'),
             (epub.Section('Simple book'),
             (c1, ))
            )

# add default NCX and Nav file
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# define CSS style
style = 'BODY {color: white;}'
nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

# add CSS file
book.add_item(nav_css)

# basic spine
book.spine = ['nav', c1]

# write to the file
epub.write_epub('test.epub', book, {})

In [ ]:
# try to programmatically generate the invisible_eink book

from ebooklib import epub

book = epub.EpubBook()

book.set_title('RY book')
book.set_language('en')

book.add_author('Raymond Yee')

# create chapter
c1 = epub.EpubHtml(title='Introduction', file_name='chap_01.xhtml', lang='en')
c1.content=u'<h1>Intro heading</h1><p>Hello world!</p>'

# add chapter
book.add_item(c1)

# define Table Of Contents
# epub.Link('chap_01.xhtml', 'Introduction'),
book.toc = (
             (epub.Section('Part 1'),
               (c1, )),
            )


# add default NCX and Nav file
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# basic spine
book.spine = ['nav', c1]

epub.write_epub("ry.epub", book, {})

      Length     Date   Time    Name
     --------    ----   ----    ----
           20  10-14-14 00:10   mimetype
         1550  10-19-14 13:05   invisible_eink_widget/content.opf
         1953  10-19-14 13:21   invisible_eink_widget/invisible_eink.js
        91556  10-04-14 12:15   invisible_eink_widget/jquery-1.6.2.min.js
          480  10-19-14 13:15   invisible_eink_widget/toc.ncx
          374  10-19-14 13:15   invisible_eink_widget/toc01.html
         2875  10-19-14 13:34   invisible_eink_widget/widget.html
          256  10-19-14 13:04   META-INF/container.xml
     --------                   -------


In [ ]:
# pieces of the invisible ebook

import ebooklib
from ebooklib import epub

book = epub.EpubBook()

book.set_title('Invisible e-ink 2')
book.set_language('en')

book.add_author('Raymond Yee')

# create chapter
# <EpubHtml:invisible_eink_widget:widget.html>
c1 = epub.EpubHtml(title='invisible eink widget', file_name='widget.html', lang='en')
c1.content = open("invisible_eink/invisible_eink.html").read()

# add chapter
book.add_item(c1)

#<EpubItem:invisible_eink_js>
#<EpubItem:invisible_eink_jquery>

c2 = epub.EpubItem(file_name='invisible_eink.js', media_type='text/javascript', 
                   content=open("invisible_eink/invisible_eink.js").read())
c3 = epub.EpubItem(file_name='jquery-1.6.2.min.js', media_type='text/javascript', 
                   content=open("invisible_eink/jquery-1.6.2.min.js").read())

book.add_item(c2)
book.add_item(c3)

# tie c2, c3 to c1 (to load scripts)
assert c2.get_type() == ebooklib.ITEM_SCRIPT
assert c3.get_type() == ebooklib.ITEM_SCRIPT

c1.add_item(c3)
c1.add_item(c2)

# make up new text

# create chapter
c4 = epub.EpubHtml(title='Intro', file_name='chap_01.xhtml', lang='en')
c4.content=u'<h1>Intro heading</h1><p class="invisible">Hello -- I should start invisibly.</p>'

# add chapter
book.add_item(c4)

c4.add_item(c3)
c4.add_item(c2)

# define Table Of Contents
# <EpubNav:invisible_eink_toc:toc01.html>

book.toc = (epub.Link('widget.html', 'Widget', 'widget'),
             (epub.Section('invisible e-ink section'),
             (c1, ))
            )

# /* Set default invisible content to have an opacity of 0 */


# define CSS style
style = """.invisible, .invisible_word {
  opacity: 0;
}"""

nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

# add CSS file to book
book.add_item(nav_css)

# add CSS file to c1

assert nav_css.get_type() == ebooklib.ITEM_STYLE

c1.add_item(nav_css)
c4.add_item(nav_css)


# add default NCX and Nav file
# <EpubNcx:invisible_eink_toc_ncx>
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# basic spine
book.spine = ['nav', c1, c4]

epub.write_epub("ry_eink.epub", book, {})